KNP解析結果から検索のための情報抽出をするノート

date: 2018/08/24

In [4]:
from knp_utils import knp_job
import pyknp
import json
import codecs

In [7]:
path_cleaned_text = "../resources/cleaned_text.json"

In [8]:
story_objects = json.loads(codecs.open(path_cleaned_text, "r", "utf-8").read())
assert isinstance(story_objects, list)

In [14]:
# KNPの解析は時間かかるので、テキスト量をしぼる
small_size_story_object = story_objects[:1]
knp_utils_input_docs = [
    {
        "text-id": story_object["title"],
        "text": story_object["body"]
    }
    for story_object in small_size_story_object
]

In [29]:
PATH_JUMAN_COMMAND = "juman"
PATH_KNP_COMMAND = "knp"
KNP_OPTIONS = "-ne-crf -tab"   # 固有名詞発見のためにCRFを呼び出し
result_obj = knp_job.main(seq_input_dict_document=knp_utils_input_docs,
                          n_jobs=-1,
                          is_normalize_text=True,
                          is_get_processed_doc=True,
                          is_split_text=True,
                          juman_command=PATH_JUMAN_COMMAND,
                          knp_command=PATH_KNP_COMMAND,
                          knp_options=KNP_OPTIONS,
                          process_mode="everytime")

In [31]:
# KNPオブジェクトを取得する
import pyknp
knp_obj = pyknp.KNP()
seq_bunsetsu_obj = [(doc_obj.sentence_index, knp_obj.result(doc_obj.parsed_result)) for doc_obj  in result_obj.seq_document_obj]

In [49]:
from pyknp.knp.tag import Tag
from typing import Optional

def find_named_entity(knp_tag_obj: Tag)->Optional[str]:
    """KNP解析結果から固有名詞を発見する関数
    
    :param knp_tag_obj: 基本句オブジェクト
    """
    if "NE" in knp_tag_obj.features:
        return knp_tag_obj.features["NE"]

for t_parsed_result in seq_bunsetsu_obj:
    for tag_obj in t_parsed_result[1].tag_list():
        if not find_named_entity(tag_obj) is None:
            print(find_named_entity(tag_obj))

LOCATION:パラオ
PERSON:繁み
PERSON:陽
PERSON:モ・バズ
LOCATION:パラオ
LOCATION:ペリリュウ島
PERCENT:半分
PERCENT:三分の二
LOCATION:カヤンガル島
LOCATION:コロール島
PERSON:尚
LOCATION:パラオ
LOCATION:マングローブ
PERSON:正覚坊
LOCATION:呉
DATE:一日
PERSON:覚しき男
DATE:一日
LOCATION:栄
LOCATION:呉
PERSON:充ちていて
PERSON:悠揚
PERSON:アア
LOCATION:オルワンガル島
LOCATION:オルワンガル島
DATE:八十年ばかり前
LOCATION:パラオ


In [60]:
# TODO 組織・団体, 動物, 植物, 自然物, 人工物-食べ物, 人工物-衣類, 人工物-乗り物, 人工物-金銭, 人工物-その他 で分類したい
from pyknp.juman.morpheme import Morpheme

# カテゴリごとに関数化するよりもforループの方が早い貴がする
def find_person_morph_category(knp_tag_obj: Tag)->Optional[Morpheme]:
    """jumanの形態素カテゴリが「人」になっている基本句を発見する
    
    :return: カテゴリが「人」である形態素
    """
    for m_obj in knp_tag_obj._mrph_list:
        if "カテゴリ:人" in m_obj.fstring:
            return m_obj
        
def find_location_morph_category(knp_tag_obj: Tag)->Optional[Morpheme]:
    """jumanの形態素カテゴリが「場所」になっている基本句を発見する
    
    :return: カテゴリが「場所」である形態素
    """
    for m_obj in knp_tag_obj._mrph_list:
        if "カテゴリ:場所" in m_obj.fstring:
            return m_obj    

        
for t_parsed_result in seq_bunsetsu_obj:
    for tag_obj in t_parsed_result[1].tag_list():
        if not find_person_morph_category(tag_obj) is None:
            print("形態素カテゴリ=人-> " + find_person_morph_category(tag_obj).repname)

        if not find_location_morph_category(tag_obj) is None:
            print("形態素カテゴリ=場所-> " + find_location_morph_category(tag_obj).repname)            

形態素カテゴリ=場所-> 島/しま
形態素カテゴリ=人-> 人/り
形態素カテゴリ=人-> 男/おとこ
形態素カテゴリ=人-> 頭/あたま
形態素カテゴリ=人-> 男/おとこ
形態素カテゴリ=人-> 衆人/しゅうじん
形態素カテゴリ=人-> 的/まと
形態素カテゴリ=人-> 男/おとこ
形態素カテゴリ=人-> 男/おとこ
形態素カテゴリ=場所-> 島/しま
形態素カテゴリ=人-> 人/ひと
形態素カテゴリ=場所-> 地方/ちほう
形態素カテゴリ=人-> 貨幣/かへい
形態素カテゴリ=人-> 宝/たから
形態素カテゴリ=人-> 男/おとこ
形態素カテゴリ=人-> 妻/つま
形態素カテゴリ=場所-> 島/しま
形態素カテゴリ=人-> 長老/ちょうろう
形態素カテゴリ=人-> 物置/ものおき
形態素カテゴリ=場所-> 舎/しゃ
形態素カテゴリ=場所-> 片隅/かたすみ
形態素カテゴリ=人-> 召し使い/めしつかい
形態素カテゴリ=場所-> 家中/いえじゅう
形態素カテゴリ=人-> 男/おとこ
形態素カテゴリ=人-> 人/り
形態素カテゴリ=人-> 怠け者/なまけもの
形態素カテゴリ=場所-> 島/しま
形態素カテゴリ=人-> 男/おとこ
形態素カテゴリ=人-> 人/り
形態素カテゴリ=人-> 槍/やり
形態素カテゴリ=人-> 盥/たらい
形態素カテゴリ=人-> 車/くるま
形態素カテゴリ=場所-> 午/うま
形態素カテゴリ=場所-> 島/しま
形態素カテゴリ=人-> 誰/だれ
形態素カテゴリ=人-> 彼/かれ
形態素カテゴリ=場所-> 床/とこ
形態素カテゴリ=人-> 男/おとこ
形態素カテゴリ=人-> 家内/かない
形態素カテゴリ=場所-> 舎/しゃ
形態素カテゴリ=人-> 縄/なわ
形態素カテゴリ=場所-> 屋根/やね
形態素カテゴリ=人-> 家具/かぐ
形態素カテゴリ=人-> 男/おとこ
形態素カテゴリ=場所-> 野/の
形態素カテゴリ=人-> 女/おんな
形態素カテゴリ=場所-> 田/た
形態素カテゴリ=場所-> 外/がい
形態素カテゴリ=人-> 男/おとこ
形態素カテゴリ=人-> 人/り
形態素カテゴリ=場所-> 西/にし
形態素カテゴリ=場所-> 海/うみ
形態素カテゴリ=人-> 男/おとこ
形態素カテゴリ=人-> 粗/あら
形態素カテゴリ=場所-> 床/とこ

In [58]:
from typing import Tuple, Dict
from pyknp.knp.pas import Pas

def find_predicate_relation(knp_tag_obj: Tag)->Optional[Tuple[Tag, Pas]]:
    """KNPの解析結果から述語と格を発見する関数

    :param knp_tag_obj: 基本句オブジェクト
    :return: タプル。タプルは基本句オブジェクトと述語項オブジェクトを持つ。そもそも述語でない場合はNone。
    """
    if knp_tag_obj.features.pas is None:
        return None
    else:
        return (knp_tag_obj, knp_tag_obj.features.pas)
        
for t_parsed_result in seq_bunsetsu_obj:
    for tag_obj in t_parsed_result[1].tag_list():
        if  find_predicate_relation(tag_obj) is not None:
            t_predicate_argument = find_predicate_relation(tag_obj)
            # ユーザーに見せる正規化代表表記
            repname = t_predicate_argument[0].repname
            # 述語を一意に識別するid
            predicate_id = t_predicate_argument[1].cfid
            # 格対応オブジェクトを得る
            dict_arguments = t_predicate_argument[1].arguments
            # 表示用の文字列
            p_case_name_relation = ""
            for case_name, seq_arg_obj in dict_arguments.items():
                assert isinstance(seq_arg_obj, list)
                seq_arg_repname = [a_obj.rep for a_obj in seq_arg_obj]
                # TODO Argumentは基本句単位でしか出力されないので、意味がある句に拡張したければ、拡張ルールを書くこと
                seq_arg_tid = [a_obj.tid for a_obj in seq_arg_obj]
                # 表示用の文字列
                p_case_name_relation += "格タイプ={}: 格表層={} / ".format(case_name, ", ".join(seq_arg_repname))
                
            print("格フレームid -> {} {}".format(predicate_id, p_case_name_relation))

格フレームid -> 極める/きわめる:動0 格タイプ=ガ: 格表層=人 / 
格フレームid -> 哀れだ/あわれだ:形3 格タイプ=ガ: 格表層=男 / 
格フレームid -> 居る/いる:動12 格タイプ=ガ: 格表層=男 / 格タイプ=時間: 格表層=昔 / 格タイプ=ニ: 格表層=島 / 
格フレームid -> 数える/かぞえる:動1 格タイプ=ヲ: 格表層=年齢 / 
格フレームid -> 不自然だ/ふしぜんだ:形3 格タイプ=ガ: 格表層=習慣 / 
格フレームid -> 無い/ない:形47 格タイプ=ガ: 格表層=習慣 / 格タイプ=ニ: 格表層=辺 / 
格フレームid -> 言える/いえる:動1 格タイプ=ガ: 格表層=こと / 格タイプ=修飾: 格表層=ハッキリ / 
格フレームid -> 若い/わかい:形9 格タイプ=ガ: 格表層=こと / 格タイプ=修飾: 格表層=余り / 
格フレームid -> 確かだ/たしかだ:形4 格タイプ=ガ: 格表層=こと / 
格フレームid -> 余る/あまる:動4 格タイプ=ガ: 格表層=毛 / 
格フレームid -> 縮れる/ちぢれる:動 
格フレームid -> 潰れる/つぶれる:動0 格タイプ=ガ: 格表層=頭 / 
格フレームid -> 成る/なる~テ形+いる/いる:動2 格タイプ=ガ: 格表層=貌 / 格タイプ=ト: 格表層=的 / 
格フレームid -> 薄い/うすい:形3 格タイプ=ガ: 格表層=脣 / 
格フレームid -> 見事だ/みごとだ:形12 格タイプ=ガ: 格表層=黒檀 / 
格フレームid -> 様/よう:判1 格タイプ=ガ: 格表層=艶 / 
格フレームid -> 無い/ない:形182 格タイプ=ガ: 格表層=艶 / 格タイプ=外の関係: 格表層=こと / 格タイプ=ニ: 格表層=顔色 / 
格フレームid -> 甚だしい/はなはだしい:形1 格タイプ=ガ: 格表層=もの / 格タイプ=修飾: 格表層=一層 / 
格フレームid -> する/する~テ形+いる/いる:動2 格タイプ=ガ: 格表層=こと / 格タイプ=ヲ: 格表層=さ / 格タイプ=ニ: 格表層=もの / 格タイプ=修飾: 格表層=おまけに / 
格フレームid -> 人/ひと:判1 格タイプ=ガ: 格表層=男 / 

In [ ]:
# TODO 格が特定のNamed entity/人/場所であることを判定したい
# TODO 基本句idで紐付けをして、Named entity/人/場所
# TODO CRFで判定した情報とJumanが判定した情報は別にしておきたい。階層構造にしておく人-crf, 人-jumanなど
